# MAP 433
## Groupe 2
### Exploration Numérique 2

COSTA, Caio; SILVA CLAUDINO, Ariel; ZUIN RUIZ, Luis Henrique

### Question 1


La fonction de vraisemblance du modèle se présente ainsi :

￼
$$L_{X,Y}(\theta) = L_X(\mu_0, \sigma_0^2) \cdot L_Y(\mu_1, \sigma_1^2) \Rightarrow \\
L_{X,Y}(\theta) = (2\pi\sigma_0^2)^{-\frac{n}{2}}(2\pi\sigma_1^2)^{-\frac{p}{2}} \exp\left(-\frac{\sum_{i=1}^{n}(X_i - \mu_0)^2}{2\sigma_0^2} - \frac{\sum_{j=1}^{p}(Y_j - \mu_1)^2}{2\sigma_1^2}\right)$$

où ￼ est la vraisemblance associée aux observations $( X_1, \dots, X_n )$ et ￼ correspond à celle des observations $( Y_1, \dots, Y_p )$.

Comme le premier terme dépend uniquement de  $(\mu_0, \sigma_0^2)$  et le second uniquement de $ (\mu_1, \sigma_1^2) $￼, il est possible de maximiser chaque partie séparément afin de trouver le maximum de vraisemblance pour $\theta$￼. En nous appuyant sur des résultats antérieurs, nous savons que l’estimateur du maximum de vraisemblance pour des observations i.i.d. $( X_1, \dots, X_n )$ sous un modèle  $\mathcal{N}(\mu, \sigma^2)$ ￼ est donné par :

$$(\hat{\mu}, \hat{\sigma}^2) = \left(\frac{1}{n}\sum_{i=1}^{n} X_i, \frac{1}{n} \sum_{i=1}^{n}(X_i - \hat{\mu})^2 \right)$$

￼
Par conséquent, on en déduit que l’estimateur du maximum de vraisemblance de $\theta$ est :

$$
\hat{\theta} = (\hat{\mu}_0, \hat{\mu}_1, \hat{\sigma}_0^2, \hat{\sigma}_1^2) \Rightarrow\\
\boxed{\hat{\theta} =\left(\frac{1}{n} \sum_{i=1}^{n} X_i, \frac{1}{p} \sum_{j=1}^{p} Y_j, \frac{1}{n} \sum_{i=1}^{n}(X_i - \hat{\mu}_0)^2, \frac{1}{p} \sum_{j=1}^{p}(Y_j - \hat{\mu}_1)^2 \right)}
$$


### Question 2

On définit la statistique $ F(z) = \frac{n(p-1)\hat{\sigma}_0^2}{p(n-1)\hat{\sigma}_1^2} $ 

Sous $ H_0 \left( \sigma_0^2 = \sigma_1^2 \right) $, nous avons (Définition IV-3.26 du poly) :  
$$
F \sim \mathcal{F}isher(n-1, p-1)
$$

Donc, la zone de rejet de niveau $\alpha$ est :  
$$
R_\alpha = \left\{ z \in \mathbb{R}^{n+p} \middle| F < q_{\alpha/2}^{(n-1, p-1)} \quad ou \quad F > q_{1-\alpha/2}^{(n-1, p-1)} \right\}
$$

où $q_{\gamma}^{(n-1, p-1)}$ est le quantile d'ordre $\gamma$ d'une loi de Fisher de paramètres $(n-1, p-1)$.


Alors, sachant que le test peut être défini par la zone de rejet, on obtient :  
$$
\left\{ z ~\middle|~
    \frac{n(p-1)\hat{\sigma}_0^2}{p(n-1)\hat{\sigma}_1^2} < q_{\alpha/2}^{(n-1, p-1)}
    \quad \text{ou} \quad
    \frac{n(p-1)\hat{\sigma}_0^2}{p(n-1)\hat{\sigma}_1^2} >  q_{1-\alpha/2}^{(n-1, p-1)}
\right\}
$$

Le test est défini par :  
$$
\phi_{\alpha}(X,Y) = \mathbb{1}_{\{(X,Y) \in R_\alpha\}}(X,Y)
$$

### Question 3

In [20]:
import numpy as np
from abc import ABC, abstractmethod
import scipy.stats as stats

# Data for each column
x = [13, 130, 39, 33, 10, 13, 68, 18, 3, 11, 38, 23, 60, 5, 9, 59, 5, 86, 22, 70, 58, 3, 167, 15, 30, 8, 20, 67, 26, 19]
y = [14, 8, 20, 3, 138, 122, 78, 69, 111, 3, 128, 31, 18, 35, 111, 109, 36, 27, 32, 35, 12, 27, 8, 3, 80, 91, 68, 66, 176, 15]

class TestPhi(ABC):
    def __init__(self, x, y, err: float = 0.0001):
        self.x = x
        self.y = y
        self.n = len(x)
        self.p = len(y)
        self.err = err

    @abstractmethod
    def test_alpha(self, alpha: float) -> bool:
        # Returns 0 if approved or 1 if repproved
        if (alpha < 0) or (alpha >= 1):
            raise ValueError("Alpha out of interval")
        return True
        
    def find_p_value_rec(self, lim_inf:float, lim_sup:float) -> float:
        if lim_sup < lim_inf:
            raise ValueError("lim_sup should always be greater than lim_inf")
        if (not self.test_alpha(lim_inf)) and (self.test_alpha(lim_sup)):
            middle_value = (lim_sup + lim_inf)/2
            if (np.abs(lim_sup - lim_inf) < self.err):
                return middle_value
            if (self.test_alpha(middle_value)):
                # Middle test fails, just like for lim_sup.
                # It then becomes the new lim_sup
                return self.find_p_value_rec(lim_inf, middle_value)
            # Middle test succeeds, just like for lim_inf.
            # It then becomes the new lim_inf
            return self.find_p_value_rec(middle_value, lim_sup)

        raise ValueError("p-value not found")


    def find_p_value(self) -> float:
        lim_inf = 0
        lim_sup = 1 - self.err

        if (self.test_alpha(lim_inf)):
            raise ValueError("lim_inf should accept all tests")
        
        if (not self.test_alpha(lim_sup)):
            return lim_sup
        
        return self.find_p_value_rec(lim_inf, lim_sup)

        

class TestQ3(TestPhi):
    def __init__(self, x, y, err: float = 0.0001):
        super().__init__(x, y, err)
        self.set_test_pivotale_func()
    
    def set_test_pivotale_func(self) -> None:
        mu_x = np.average(self.x)
        mu_y = np.average(self.y)

        s0_2 = np.sum(np.power(self.x - mu_x, 2))/(self.n -1)
        s1_2 = np.sum(np.power(self.y - mu_y, 2))/(self.p -1)

        self.test_const = s0_2/s1_2


    
    def get_quantils(self, alpha) -> float:
        q_1_alpha_2 = stats.f.ppf(1 - alpha/2, self.n-1, self.p-1)
        q_alpha_2 = stats.f.ppf(alpha/2, self.n-1, self.p-1)
        return [q_alpha_2, q_1_alpha_2]
    
    
    def test_alpha(self, alpha: float) -> bool:
        quantils = self.get_quantils(alpha)
        return  (self.test_const < quantils[0]) or (self.test_const > quantils[1])
    
test_q3 = TestQ3(x, y)
p_value_q3 = test_q3.find_p_value()
print(f"The calculated p_value for the test was: {p_value_q3}")

The calculated p_value for the test was: 0.22895049133300785


### Question 4

On sait que $\hat{\mu}_0$ et $\hat{\mu}_1$ sont des estimateurs indépendants avec les distributions suivantes :
$ \hat{\mu}_0 \sim \mathcal{N}\left(\mu_0, \frac{\sigma^2}{n}\right)\quad$ et $\quad \hat{\mu}_1 \sim \mathcal{N}\left(\mu_1, \frac{\sigma^2}{p}\right) $

En conséquence, la différence $\hat{\mu}_0 - \hat{\mu}_1$ suit la loi normale :

$$\hat{\mu}_0 - \hat{\mu}_1 \sim \mathcal{N}\left(\mu_0 - \mu_1, \frac{p + n}{np}\sigma^2\right)$$


Nous pouvons alors en déduire que :
$$
\frac{1}{\sigma}\sqrt{\frac{np}{p + n}}\left[(\hat{\mu}_0 - \hat{\mu}_1) - (\mu_0 - \mu_1)\right] \sim \mathcal{N}(0, 1)
$$

Par ailleurs, on définit $S_{n,p}$ comme l'estimateur corrigé de l'écart-type :
$$S_{n,p} = \sqrt{\frac{n+p}{n+p-2}\hat{\sigma}^2}$$
et la statistique suivante suit une loi du khi-carré :
$$
(n+p-2)\frac{S_{n,p}^2}{\sigma^2} \sim \chi^2(n+p-2)
$$

Finalement, on obtient une statistique de Student :
$$
\frac{1}{S_{n,p}}\sqrt{\frac{np}{p + n}}\left[(\hat{\mu}_0 - \hat{\mu}_1) - (\mu_0 - \mu_1)\right] \sim \mathcal{T}(n+p-2)
$$

On en déduit donc que la probabilité suivante est égale à $1 - \alpha$ :
$$
P_{\theta}\left(q_{\alpha/2} \leq \frac{1}{S_{n,p}}\sqrt{\frac{np}{p + n}}\left[(\hat{\mu}_0 - \hat{\mu}_1) - (\mu_0 - \mu_1)\right] \leq q_{1 - \alpha/2}\right) = 1 - \alpha
$$
où $q_\alpha$ est le quantile d'ordre $\alpha$ de la loi de Student avec $n + p - 2$ degrés de liberté.

L'intervalle de confiance à niveau $\alpha$ est donc donné par :
$$
\boxed{\mathcal{I}\alpha(X,Y) = \left[(\hat{\mu}_0 - \hat{\mu}_1) - q_{1 - \alpha/2}S_{n,p}\sqrt{\frac{p + n}{np}}; (\hat{\mu}_0 - \hat{\mu}_1) + q_{1 - \alpha/2}S_{n,p}\sqrt{\frac{p + n}{np}}\right]}
$$

### Question 5.

On définit la région de rejet $R_\alpha$  de la manière suivante :

$$
R_\alpha = \left\{ z \in \mathbb{R}^{n+p} \Bigg| \frac{|\hat{\mu}_0 - \hat{\mu}_1|}{S_{n,p}} \sqrt{\frac{np}{p+n}} > q_{1-\alpha/2} \right\}
$$

où $q_\alpha$ est le quantile d'ordre $\alpha$ de la loi de Student avec $n + p - 2$ degrés de liberté.

On note :

$$\Theta_0 = \{(\mu, \mu, \sigma^2)\} \quad \text{et} \quad \Theta_1 = \{(\mu_0, \mu_1, \sigma^2) \mid \mu_0 \neq \mu_1\}$$

Ainsi, pour tout  $\theta_0 \in \Theta_0$  :


$$P_{\theta_0}((X, Y) \in R_\alpha) = P_{\theta_0} \left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p+n}} < -q_{1-\alpha/2} \right) + 1 - P_{\theta_0} \left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p+n}} \leq q_{1-\alpha/2} \right)$$

On utilise le fait que la loi de Student est symétrique, et que sous $H_0$ , on a  $\mu_0 - \mu_1 = 0$ . On obtient alors :

$$\boxed{P_{\theta_0}((X, Y) \in R_\alpha) = P_{\theta_0} \left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p+n}} < q_{\alpha/2} \right) + \left( 1 - \left( 1 - \frac{\alpha}{2} \right) \right) = \alpha}$$

Finalement, on obtient le test  $\phi(X, Y) = \mathbf{1}_{(X, Y) \in R_\alpha}$, qui est de taille $\alpha$, pour les hypothèses données.

### Question 6

La p-valeur est donnée par :
\begin{align*}
    \hat{\alpha}(X,Y) &= \inf \left\{ \alpha \in [0,1] \ \middle| \ \frac{|\hat{\mu}_0 - \hat{\mu}_1|}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha/2} \right\} \\
\end{align*}

Nous allons, donc, calculer cet inf ci-dessus numériquement avec le code ci-dessous:

In [30]:
class TestQ6(TestPhi):
    def __init__(self, x, y, err: float = 0.0001):
        super().__init__(x, y, err)
        self.set_test_pivotale_func()
    
    def set_test_pivotale_func(self) -> None:
        mu_x = np.average(self.x)
        mu_y = np.average(self.y)
        sigma2 = (np.sum(np.power(self.x - mu_x, 2)) + np.sum(np.power(self.y - mu_y, 2)))/(self.n + self.p)
        s2np = sigma2*(self.n + self.p)/(self.n + self.p-2)
        self.test_const = (np.sqrt(self.n*self.p/(self.n+self.p)))*np.abs(mu_x - mu_y)/np.sqrt(s2np)

    def get_quantil(self, alpha) -> float:
        deg_freed = self.n + self.p - 2
        q_1_alpha_2 = stats.t.ppf(1 - alpha/2, deg_freed)
        return q_1_alpha_2
    
    
    def test_alpha(self, alpha: float) -> bool:
        return  self.test_const >= self.get_quantil(alpha)




test_q6 = TestQ6(x, y)
p_value_q8 = test_q6.find_p_value()
print(f"The calculated p_value for the test was: {p_value_q8}")

The calculated p_value for the test was: 0.1117747100830078


### Question 7
Il s’agit simplement de faire une petite modification du test précédent. On définit alors :
$$
\mathcal{R}_\alpha = \left\{ z \in \mathbb{R}^{n+p} \ \middle| \ \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right\}
$$

Nous allons maintenant démontrer que :
$$
\sup_{\theta \in \Theta_0} P_\theta\left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right) = \alpha
$$
où : $\Theta_0 = \left\{ (\mu_0, \mu_1, \sigma^2) \ | \ \mu_0 \leq \mu_1 \right\}$.

Or, sous l’hypothèse  $H_0$ , nous avons :
$$
\begin{align*}
P_\theta\left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right) &= P_\theta\left( \frac{(\hat{\mu}_0 - \hat{\mu}_1) - (\mu_0 - \mu_1)}{S_{n,p}} \sqrt{\frac{np}{p + n}} + \frac{\mu_0 - \mu_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right) \\
&\leq P_\theta\left( \frac{(\hat{\mu}0 - \hat{\mu}1) - (\mu_0 - \mu_1)}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right) = \alpha
\end{align*}
$$
en utilisant le fait que $\frac{1}{S_{n,p}} \sqrt{\frac{np}{p + n}} \left[ (\hat{\mu}0 - \hat{\mu}1) - (\mu_0 - \mu_1) \right] \sim \text{Student}(n + p - 2)$ et que, sous  $H_0$ , $\mu_0 - \mu_1 \leq 0$, ce qui implique que $\frac{\mu_0 - \mu_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} \leq 0$  car  $S_{n,p} > 0$  presque sûrement.

Ainsi, nous avons montré que :

$\sup_{\theta \in \Theta_0} P_\theta\left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right) \leq \alpha$


De plus, pour $\theta \in \Theta_0$ tel que $\mu_0 = \mu_1$, on a :
$$
\boxed{P_\theta\left( \frac{\hat{\mu}_0 - \hat{\mu}_1}{S_{n,p}} \sqrt{\frac{np}{p + n}} > q_{1-\alpha} \right) = \alpha}
$$
￼


### Question 8

In [21]:
class TestQ8(TestPhi):
    def __init__(self, x, y, err: float = 0.0001):
        super().__init__(x, y, err)
        self.set_test_pivotale_func()
    
    def set_test_pivotale_func(self) -> None:
        mu_x = np.average(self.x)
        mu_y = np.average(self.y)
        sigma2 = (np.sum(np.power(self.x - mu_x, 2)) + np.sum(np.power(self.y - mu_y, 2)))/(self.n + self.p)
        s2np = sigma2*(self.n + self.p)/(self.n + self.p-2)
        self.test_const = (np.sqrt(self.n*self.p/(self.n+self.p)))*(mu_x - mu_y)/np.sqrt(s2np)

    def get_quantil(self, alpha) -> float:
        deg_freed = self.n + self.p - 2
        q_1_alpha_2 = stats.t.ppf(1 - alpha, deg_freed)
        return q_1_alpha_2
    
    
    def test_alpha(self, alpha: float) -> bool:
        return  self.test_const >= self.get_quantil(alpha)




test_q8 = TestQ8(x, y)
p_value_q8 = test_q8.find_p_value()
print(f"The calculated p_value for the test was: {p_value_q8}")

The calculated p_value for the test was: 0.9441499603271484
